In [18]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin
import os

def web_crawler(url, max_depth):
    visited_urls = set()

    def crawl(url, depth):
        if depth > max_depth:
            return

        print("Crawling:", url)

        try:
            response = requests.get(url)
        except requests.exceptions.RequestException:
            # Handle any exceptions that occur during the request
            return

        visited_urls.add(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract all links on the page
        for link in soup.find_all('a'):
            href = link.get('href')
            if href and href.startswith('http') and href not in visited_urls:
                crawl(href, depth + 1)

        # Download MP3 files
        for file_link in soup.find_all('a'):
            href = file_link.get('href')
            if href and href.endswith('.mp3'):
                file_url = urljoin(url, href)
                download_file(file_url)

    def download_file(url):
        try:
            response = requests.get(url)
            filename = os.path.basename(urlparse(url).path)
            file_path = os.path.join("mp3_files", filename)

            if os.path.isfile(file_path):
                return

            with open(file_path, 'wb') as file:
                file.write(response.content)
        except requests.exceptions.RequestException:
            # Handle any exceptions that occur during the file download
            return

    crawl(url, 0)

# Specify the destination directory here
os.makedirs("mp3_files", exist_ok=True)

# Example usage
web_crawler('http://accent.gmu.edu/soundtracks/', 2)


Crawling: http://accent.gmu.edu/soundtracks/
